In [1]:
import pandas as pd
df = pd.read_excel("Z:/完成檔案/black & law/law_清理過3.xlsx")

In [2]:
df.head()

,Id,court,number,date,reason,content,判決
0,726,臺灣台北地方法院 裁判書 -- 民事類,臺灣台北地方法院 103 年婚字第 390 號民事判決,105.02.19,離婚等,臺灣臺北地方法院民事判決103年度婚字第390號原告蕭永崑訴訟代理人連1鴻律師被告彭宜慧訴訟...,駁回
1,727,臺灣台北地方法院 裁判書 -- 民事類,臺灣台北地方法院 104 年婚字第 236 號民事其他,104.10.30,離婚,"臺灣臺北地方法院裁判書--民事類【裁判字號】104,婚,236【裁判日期】1041030【裁...",駁回
2,728,臺灣台北地方法院 裁判書 -- 民事類,臺灣台北地方法院 103 年婚字第 229 號民事判決,104.10.23,離婚等,臺灣臺北地方法院民事判決103年度婚字第229號原告林貞靜被告晏世豪上列當事人間請求離婚等事...,准許
3,729,臺灣台北地方法院 裁判書 -- 民事類,臺灣台北地方法院 103 年婚字第 386 號民事判決,104.04.10,離婚等,臺灣臺北地方法院民事判決103年度婚字第386號原告顏承濬訴訟代理人吳妙白律師被告顏耶咪（E...,准許
4,730,臺灣台北地方法院 裁判書 -- 民事類,臺灣台北地方法院 102 年婚字第 351 號民事其他,104.01.30,離婚等,"臺灣臺北地方法院裁判書--民事類【裁判字號】102,婚,351【裁判日期】1040130【裁...",准許


In [7]:
# 一個以上空白還沒處理
df['content'] = df['content'].str.replace(" ","")
df['content'] = df['content'].str.replace("一","1").str.replace("二","2").str.replace("三","3").str.replace("四","4").str.replace("五","5").str.replace("六","6").str.replace("七","7").str.replace("八","8").str.replace("九","9").str.replace("零","0").str.replace("○","0").str.replace("１","1").str.replace("５",'5').str.replace("０",'0')
df['content'] = df['content'].str.replace("千","").str.replace("十","").str.replace("\n","").str.replace("\u3000","").replace('及',"")

In [8]:
df2 = df[df['判決']=="准許"]

In [9]:
import re
regex_new = re.compile('([民同]法第1052條第\d(項|款)((第?\d+(?:、\d+)*款?)|以外之重大事由)?)')

In [10]:
for i in df[df['Id']==726]['content']:
    match = re.findall(regex_new,i)
print(match)

[('民法第1052條第1項第3款', '項', '第3款', '第3款'), ('民法第1052條第1項第3款', '項', '第3款', '第3款'), ('民法第1052條第2項', '項', '', ''), ('民法第1052條第1項第3款', '項', '第3款', '第3款')]


### find_all找尋所有1052法條

In [11]:
law, law_id = [], []
for i,k in list(zip(df2['content'],df2['Id'])):
    law_temp = []
    match_new = re.findall(regex_new,i)
    for a in match_new:
        for b in a:
            if '1052' in b:
                law_temp.append(b)
    law.append(law_temp)
    law_id.append(k)
    

In [12]:
law1052 = list(zip(law, law_id))

In [15]:
law1052[0:3]

[(['民法第1052條第1項第3款', '民法第1052條第1項第3款', '民法第1052條第1項第3款'], 728),
 (['民法第1052條第2項', '民法第1052條第1項以外之重大事由', '民法第1052條第2項', '民法第1052條第2項'], 729),
 (['民法第1052條第1項第2款',
   '民法第1052條第1項第3款',
   '民法第1052條第1項以外之重大事由',
   '民法第1052條第2項',
   '民法第1052條第2項',
   '民法第1052條第2項',
   '民法第1052條第1項第2款',
   '民法第1052條第1項第3款',
   '民法第1052條第2項'],
  730)]

In [75]:
for i in set(law1052[1][0]):
    print(i)

民法第1052條第2項
民法第1052條第1項以外之重大事由


In [71]:
("判准" or "准許") in df2['content'][3].split('民法第1052條第2項')[-1]

True

In [98]:
# 先依順序set那一篇出現過的法條
# 再依序split每一個法條，並找尋最後一個被切出的段落
# 在看判准or准許兩詞有無出現在最後一個段落
number = 0
correct, ID = [], []
for i,k in list(zip(df2['content'],df2['Id'])):
    # 如果set後只有一條法案，合理推測那一條就是判決准許的法案
    if len(set(law1052[number][0])) == 1:
        correct.append(set(law1052[number][0]))
    elif len(set(law1052[number][0])) > 1:
        # 依序切開法條
        a = []
        for z in set(law1052[number][0]):
            if ("判准" or "准許") in i.split(z)[-1] == True: # i是文章，z是法條
                a.append(z)
            else:
                pass
        try:
            correct.append(a[-1])
        except:
            correct.append(None)
        ID.append(k)
    number += 1
final = list(zip(correct, ID))

In [99]:
final

[({'民法第1052條第1項第3款'}, 729),
 (None, 730),
 (None, 731),
 (None, 732),
 (None, 733),
 (None, 735),
 (None, 736),
 (None, 737),
 (None, 738),
 (None, 739),
 (None, 740),
 (None, 741),
 (None, 747),
 ({'民法第1052條第1項第3款'}, 748),
 ({'民法第1052條第2項'}, 749),
 (None, 750),
 (None, 752),
 (None, 755),
 (None, 756),
 ({'民法第1052條第1項第3款'}, 759),
 (None, 764),
 (None, 767),
 (None, 768),
 ({'民法第1052條第2項'}, 769),
 (None, 770),
 ({'民法第1052條第1項第3款'}, 772),
 (None, 773),
 (None, 775),
 (None, 776),
 (None, 777),
 (None, 778),
 (None, 779),
 (None, 781),
 (None, 783),
 (None, 790),
 (None, 791),
 (None, 793),
 (None, 794),
 ({'民法第1052條第2項'}, 795),
 (None, 798),
 (None, 809),
 ({'民法第1052條第2項'}, 810),
 ({'民法第1052條第1項第5款'}, 811),
 ({'民法第1052條第1項第3款'}, 812),
 (None, 814),
 (None, 815),
 (None, 816),
 (None, 820),
 (None, 822),
 ({'民法第1052條第1項第3款'}, 824),
 ({'民法第1052條第1項第3款'}, 825),
 (None, 826),
 ({'民法第1052條第1項第3款'}, 827),
 ({'民法第1052條第1項第3款'}, 828),
 ({'民法第1052條第1項第3款'}, 832),
 (None, 833),
 (None, 834),
 (No